# DDoS dataset analysis

L'obiettivo di questo progetto è analizzare un dataset contenente informazioni relative sia ad attacchi DDoS che a normale traffico.
Questo per poter realizzare un'applicazione capace di distinguere il traffico sospetto da quello legittimo e poter quindi tempestivamente bloccare i tentativi di attacco.

## Descrizione del dataset

Il dataset è composto da una serie di campionamenti, ognuno dei quali fa riferimento ad una specifica connessione tra due macchine. 
Un campionamento, oltre ad identificare le due macchine e le interfacce di rete coinvolte nella comunicazione, contiene altre informazioni aggregate sullo stato della connessione stessa, come ad esempio il numero di pacchetti e di byte scambiati durante il periodo di campionamento.

Il dataset si può trovare a questo [link](https://www.kaggle.com/datasets/yahyaalhaj/biotddos).
Si compone di tre file, che possono essere scaricati come un unico file "zip" a questo [link](https://www.kaggle.com/datasets/yahyaalhaj/biotddos/download). 

### Descrizione dei file

Tutti e tre i file hanno lo stesso formato e contengono tre diverse categorie di attacchi, ovvero attacchi "TCP", "UDP" e "HTTP".
Non siamo interessati a tenere conto di questa distinzione, perciò consideriamo questi file come contenenti lo stesso tipo di dati.
Ogni file è in formato CSV e contiene un insieme di record, ognuno composto dai valori:
* "stime": la data e l'ora di inizio del campionamento in formato UNIX timestamp, dotato anche dei millisecondi 
* "flgs": le "flag" TCP e UDP osservate nei pacchetti che sono stati scambiati durante il campionamento
* "proto": il protocollo di livello di rete utilizzato dai pacchetti che sono stati scambiati durante il campionamento
* "saddr": l'indirizzo IP della sorgente della connessione che è interessata dal campionamento
* "sport": la porta della sorgente della connessione che è interessata dal campionamento
* "dir": la direzione del flusso dati
* "daddr": l'indirizzo IP della destinazione della connessione che è interessata dal campionamento
* "dport": la porta della destinazione della connessione che è interessata dal campionamento
* "pkts": il numero di pacchetti scambiati durante questo campionamento
* "bytes": il numero di byte scambiati durante questo campionamento
* "state": lo stato della connessione 
* srcid
* ltime
* seq
* dur
* mean
* stddev
* smac
* dmac
* sum
* min
* max
* soui
* doui
* sco
* dco
* spkts
* dpkts
* sbytes
* dbytes
* rate
* srate
* drate
* record
* attack
* category
* subcategory

In [2]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1655456610689_0004,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1655456610689_0004,spark,idle,Link,Link,✔


In [7]:
val bucketName = "unibo-bd2122-mcastellucci/project"

val pathTCPDataset = s"s3a://$bucketName/DDoS_TCP.csv"
val pathUDPDataset = s"s3a://$bucketName/DDoS_UDP.csv"
val pathHTTPDataset = s"s3a://$bucketName/DDoS_HTTP.csv"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketName: String = unibo-bd2122-mcastellucci/project
pathTCPDataset: String = s3a://unibo-bd2122-mcastellucci/project/DDoS_TCP.csv
pathUDPDataset: String = s3a://unibo-bd2122-mcastellucci/project/DDoS_UDP.csv
pathHTTPDataset: String = s3a://unibo-bd2122-mcastellucci/project/DDoS_HTTP.csv
res8: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1655456610689_0004/


In [8]:
val dataset = sc.textFile(s"$pathTCPDataset,$pathUDPDataset,$pathHTTPDataset")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dataset: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2122-mcastellucci/project/DDoS_TCP.csv,s3a://unibo-bd2122-mcastellucci/project/DDoS_UDP.csv,s3a://unibo-bd2122-mcastellucci/project/DDoS_HTTP.csv MapPartitionsRDD[3] at textFile at <console>:31


In [9]:
dataset.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res9: Array[String] = Array("stime";"flgs";"proto";"saddr";"sport";"dir";"daddr";"dport";"pkts";"bytes";"state";"srcid";"ltime";"seq";"dur";"mean";"stddev";"smac";"dmac";"sum";"min";"max";"soui";"doui";"sco";"dco";"spkts";"dpkts";"sbytes";"dbytes";"rate";"srate";"drate";"record";"attack";"category";"subcategory")
